# Initializing

## Importing libraries

In [2]:
import numpy as np
import pandas as pd

## Importing SIOPs

The SIOPs are stored in csv files - we need to read those in using pandas:

In [3]:
siops_abc = pd.read_csv('data/siops_abc.csv') #provide path to the file
siops_bb = pd.read_csv('data/siops_bb.csv') #provide path to the file

In [4]:
siops_abc

,Wavelength,a*chl,a*mss,a*cdom,b*chl,b*mss
0,400,0.052899,0.080000,1.607792,0.12,0.4
1,402,0.053036,0.078450,1.569649,0.12,0.4
2,404,0.054574,0.076930,1.532410,0.12,0.4
3,406,0.055862,0.075495,1.497387,0.12,0.4
4,408,0.057642,0.074032,1.461863,0.12,0.4
...,...,...,...,...,...,...
171,742,-0.000148,0.002939,0.027872,0.12,0.4
172,744,-0.000032,0.002882,0.027211,0.12,0.4
173,746,-0.000183,0.002826,0.026565,0.12,0.4
174,748,0.000028,0.002773,0.025958,0.12,0.4


In [5]:
siops_bb

,Wave,bb*chl,bb*mss
0,400,0.001715,0.016487
1,405,0.001700,0.016420
2,410,0.001685,0.016355
3,415,0.001671,0.016290
4,420,0.001656,0.016227
...,...,...,...
66,730,0.001117,0.013555
67,735,0.001111,0.013525
68,740,0.001106,0.013495
69,745,0.001101,0.013466


# Bio-optical model: generate IOPs

We're going to generate IOPs for a range of concentrations for each optically significant material (OSM). In this case, our OSMs are phytoplankton chlorophyll-a (chl), mineral suspended sediments (MSS) and coloured dissolved organic matter (CDOM).

Here we're doing equally spaced concentrations for a given range of each material, giving:

- Chl = 1 and 10 mg/m^3
- MSS = 1, 5.5 and 10 g/m^3
- CDOM = 0.01, 0.173, 0.337, and 0.5 m^-1

In [6]:
nchl = 2
nmss = 3
ncdom = 4 

chl = np.linspace(1.,10.,nchl)
mss = np.linspace(1.,10.,nmss)
cdom = np.linspace(0.01,0.5,ncdom)

What does one of these arrays look like? It's a 1-dimensional array:

In [7]:
mss

array([ 1. ,  5.5, 10. ])

Another way of specifying constituent concentrations in an array:

In [8]:
PIC = np.array([0.1, 0.2])

In [9]:
PIC

array([0.1, 0.2])

## OSM concentrations and SIOPs to IOPs?

The total absorption, $a$, of seawater can be written as:

$$a = a_w + a_{chl} + a_{mss} + a_{cdom}$$
$$a = a_w + [CHL]a*_{chl} + [MSS]a*_{mss} + [CDOM]a*_{cdom}$$

where:
- $a_{chl}$ = chlorophyll-a absorption
- $a_{mss}$ = MSS absorption
- $a_{cdom}$ = CDOM absorption
- $[CHL]$ = chlorophyll-a concentration
- $[MSS]$ = MSS concentration
- $[CDOM]$ = CDOM concentration
- $a*_{chl}$ = chlorophyll-a specific absorption
- $a*_{mss}$ = MSS specific absorption
- $a*_{cdom}$ = CDOM specific absorption
- $a_{w}$ = water absorption

[See the IOCCG Report 3, section 2.6](https://ioccg.org/wp-content/uploads/2015/10/ioccg-report-03.pdf)

A similar equation can be written for attenuation ($c$) and backscattering ($b_b$), except CDOM does not scatter and so is not included when calculating $b_b$.

## MSS and CDOM absorption coefficients following the above

The OSM concentrations are saved as 1-D arrays:

In [10]:
mss.shape

(3,)

But we need them to be 2-D arrays to multiply with the MSS SIOP ($a*_{mss}$) - and similarly for CDOM

In [11]:
mss2d = np.expand_dims(mss, axis = 0)
cdom2d = np.expand_dims(cdom, axis = 0)

And then we can multiply by the SIOPs to calculate the absorption for each material:

In [12]:
amss = np.expand_dims(siops_abc['a*mss'],axis=1) * mss2d
acdom = np.expand_dims(siops_abc['a*cdom'],axis=1) * cdom2d

And the shapes of these arrays?

In [13]:
amss.shape

(176, 3)

In [14]:
acdom.shape

(176, 4)

So we have one column for each concentration, and each row is a different wavelength. 

## Phytoplankton absorption coefficients calculated using [Bricaud et al (1998)](https://agupubs.onlinelibrary.wiley.com/doi/pdfdirect/10.1029/98JC02712).

The Bricaud et al model for calculating phytoplankton absorption is a little different from the above. First it calculates the chl absorption at 440 nm as follows:

$$a_{chl}(440) = 0.05[CHL]^{0.626}$$

Then it multiplies this by the chlorophyll specific absorption *normalized to 440 nm* - this means we have to divide the chlorophyll specific absorption at all wavelengths by the chlorophyll specific absorption at 440 nm.

Calculating the normalized chlorophyll specific absorption coefficient:

In [15]:
achlplus = siops_abc['a*chl'] / siops_abc['a*chl'].loc[siops_abc['Wavelength'] == 440].iloc[0]

And $a_{chl}$ at 440 nm:

In [16]:
achl_440 = 0.05 * (chl**0.626)

Converting to numpy arrays and expanding to two dimensions (as for CDOM and MSS)

In [17]:
achl_440 = np.expand_dims(achl_440, axis=0)
achlplus = np.expand_dims(achlplus, axis=1)

Broadcasting numpy arrays:

In [18]:
achl = achl_440*achlplus

What is the final shape of this array?

In [19]:
achl.shape

(176, 2)

So we have one column for each chlorophyll concentration, and each row is a different wavelength.

## Total nonwater absorption coefficents

We need to make copies of our achl, acdom and amss arrays, so we can add together all the combinations of constituents

| chl | mss | cdom|
|---|---|---|
| 1 | 1 | 0.01 |
| 1 | 1 | 0.17 |
| 1 | 1 | 0.34 |
| 1 | 1 | 0.5 |
| 1 | 5.5 | 0.01 |
| 1 | 5.5 | 0.17 |
| 1 | 5.5 | 0.34 |
| 1 | 5.5 | 0.5 |
| 1 | 10 | 0.01 |
| 1 | 10 | 0.17 |
| 1 | 10 | 0.34 |
| 1 | 10 | 0.5 |
| 10 | 1 | 0.01 |
| 10 | 1 | 0.17 |
| 10 | 1 | 0.34 |
| 10 | 1 | 0.5 |
| 10 | 5.5 | 0.01 |
| 10 | 5.5 | 0.17 |
| 10 | 5.5 | 0.34 |
| 10 | 5.5 | 0.5 |
| 10 | 10 | 0.01 |
| 10 | 10 | 0.17 |
| 10 | 10 | 0.34 |
| 10 | 10 | 0.5 |

First we want to repeat the `achl` array i.e. we want to repeat each column 12 times. Why 12? See the table above - we want to add the first `achl` column to all possible combinations of the `amss` and `acdom` columns, which totals $3 \times 4 = 12\ $:

In [20]:
achlrep = np.repeat(achl,nmss*ncdom,axis=1)

Let's check the shape is what we expect (176, 24):

In [21]:
achlrep.shape

(176, 24)

Great! But are the columns in the right order? Let's check the first row, the first 12 columns should have the same value, and then the next 12 columns should have a different value

In [22]:
achlrep[0,:]

array([0.0345238 , 0.0345238 , 0.0345238 , 0.0345238 , 0.0345238 ,
       0.0345238 , 0.0345238 , 0.0345238 , 0.0345238 , 0.0345238 ,
       0.0345238 , 0.0345238 , 0.14592125, 0.14592125, 0.14592125,
       0.14592125, 0.14592125, 0.14592125, 0.14592125, 0.14592125,
       0.14592125, 0.14592125, 0.14592125, 0.14592125])

Great - this worked. Onto CDOM next...

For CDOM we want to tile the `acdom` array i.e. we want to make 6 copies of it, resulting in 1 array, where each set of 4 columns corresponds to a different CDOM concentration:

In [23]:
acdomrep = np.tile(acdom,nchl*nmss)

Check the shape is (176, 24):

In [24]:
acdomrep.shape

(176, 24)

And what does the first row look like?

In [25]:
acdomrep[0,:]

array([0.01607792, 0.27868391, 0.5412899 , 0.8038959 , 0.01607792,
       0.27868391, 0.5412899 , 0.8038959 , 0.01607792, 0.27868391,
       0.5412899 , 0.8038959 , 0.01607792, 0.27868391, 0.5412899 ,
       0.8038959 , 0.01607792, 0.27868391, 0.5412899 , 0.8038959 ,
       0.01607792, 0.27868391, 0.5412899 , 0.8038959 ])

And finally, MSS. Here we need to first repeat, and then tile:

In [26]:
amssrep = np.tile(np.repeat(amss,ncdom,axis=1),nchl)

And is the shape (176, 24)?

In [27]:
amssrep.shape

(176, 24)

Yes! Does the first row look like we expect? 

In [28]:
amssrep[0,:]

array([0.08, 0.08, 0.08, 0.08, 0.44, 0.44, 0.44, 0.44, 0.8 , 0.8 , 0.8 ,
       0.8 , 0.08, 0.08, 0.08, 0.08, 0.44, 0.44, 0.44, 0.44, 0.8 , 0.8 ,
       0.8 , 0.8 ])

Yes!

And the total non-water absorption is:

In [37]:
anw = achlrep + amssrep + acdomrep

## Backscattering and scattering coefficients for CHL and MSS

These are calculated using the standard model (like we did for MSS and CDOM absorption), but CDOM doesn't scatter, so isn't included here

In [30]:
chl2d = np.expand_dims(chl, axis = 0)

bmss = np.expand_dims(siops_abc['b*mss'],axis=1) * mss2d
bchl = np.expand_dims(siops_abc['b*chl'],axis=1) * chl2d

bbmss = np.expand_dims(siops_bb['bb*mss'],axis=1) * mss2d
bbchl = np.expand_dims(siops_bb['bb*chl'],axis=1) * chl2d

And then repeating the arrays to make them the right size (176,24):

In [35]:
bchlrep = np.repeat(bchl,nmss*ncdom,axis=1)
bmssrep = np.tile(np.repeat(bmss,ncdom,axis=1),nchl)

bbchlrep = np.repeat(bbchl,nmss*ncdom,axis=1)
bbmssrep = np.tile(np.repeat(bbmss,ncdom,axis=1),nchl)

In [36]:
bmssrep.shape

(176, 24)

## Nonwater backscattering and attenuation coefficients

In [40]:
bbnw = bbchlrep + bbmssrep
bnw = bchlrep + bmssrep

cnw = anw + bnw

# Create HydroLight ac, bb and input files

## Creating the wavelength arrays

In [71]:
wavelac = np.linspace(400,750,176)
wac = np.expand_dims(np.insert(wavelac,0,176), axis = 0)

wavelbb = np.linspace(400,750,71)
wbb = np.expand_dims(np.insert(wavelbb,0,71), axis = 0)

### Repeating the CHL, MSS and CDOM concentration arrays

This will be useful for writing data into the ac, bb and input HydroLight files

In [46]:
chlrep = np.repeat(chl,nmss*ncdom)
mssrep = np.tile(np.repeat(mss,ncdom,),nchl)
cdomrep = np.tile(cdom,nchl*nmss)

## Creating the ac files

The variable `path` is the full directory of where you want to save the ac files.

In [70]:
for run in range(0,nchl*nmss*ncdom,1):
    path='data/ac-files/ac'+str(run)+'.txt'
    with open(path, "w") as f:  
        aca=np.concatenate([[1],anw[:,run],cnw[:,run]])
        acb=np.concatenate([[-1],anw[:,run],cnw[:,run]])
        ac = np.array([aca, acb])
        osms = [chlrep[run], mssrep[run], cdomrep[run]]
        
        print('ac file',file=f)
        print('-----------',file=f)
        print('**need ten lines header**',file=f)
        print('10th line contains run information',file=f)
        print('11th line contains number of wavelengths & those wavelengths',file=f)
        print('12th line onwards contains ac data',file=f)
        print('ac data takes form of:',file=f)
        print('depth then a for each wavelength, then c for each wavelength',file=f)
        print('(NB - HE6 recognises end of data by negative depth)',file=f)
        print(osms,file=f)
        np.savetxt(f,wac,fmt='%i',delimiter=' ')
        np.savetxt(f,ac,fmt='%f',delimiter=' ', newline='\n')

### Creating the bb files

The variable `path` is the full directory of where you want to save the bb files.

In [78]:
for run in range(0,nchl*nmss*ncdom,1):
    path='data/bb-files/bb'+str(run)+'.txt'
    with open(path, "w") as f:  
        bba=np.concatenate([[1],bbnw[:,run]])
        bbb=np.concatenate([[-1],bbnw[:,run]])
        bb = np.array([bba, bbb])
        osms = [chlrep[run], mssrep[run], cdomrep[run]]
        
        print('bb file',file=f)
        print('-----------',file=f)
        print('**need ten lines header**',file=f)
        print('10th line contains run information',file=f)
        print('11th line contains number of wavelengths & those wavelengths',file=f)
        print('12th line onwards contains ac data',file=f)
        print('ac data takes form of:',file=f)
        print('depth then bb for each wavelength',file=f)
        print('(NB - HE6 recognises end of data by negative depth)',file=f)
        print(osms,file=f)
        np.savetxt(f,wbb,fmt='%i',delimiter=' ')
        np.savetxt(f,bb,fmt='%f',delimiter=' ', newline='\n')

### Input files

The variable `ifpath` is the full directory of where you want to save the HydroLight input files.

`acpcpath` is the full directory of where the ac files are saved - this is important because HydroLight will look there for the files.

`bbpcpath` is the full directory of where the bb files are saved - this is important because HydroLight will look there for the files.

In [75]:
ifpath = 'data/HEinputfiles/'
acpcpath = 'X:/cmitchell/projects/git-repos/lab-guides-examples-etc/HydroLight/data/ac-files/'
bbpcpath = 'X:/cmitchell/projects/git-repos/lab-guides-examples-etc/HydroLight/data/bb-files/'

In [80]:
for run in range(0,nchl*nmss*ncdom,1):
    with open(ifpath+'I_'+str(run)+'.txt', 'w') as f: #directory for where to save the input files
        print('0, 400, 700, 0.02, 488, 0.00026, 1, 5.3',file=f)
        print('description placeholder',file=f)
        print(str(run),file=f) #setting "title"
        print('0, 0, 2, 1, 0',file=f)
        print('3, 1, 0, 0, 0, 0',file=f)
        print('2, 2',file=f)
        print('0, 0,',file=f)
        print('0, 1, 440, 1, 0.014',file=f)
        print('2, -666, 440, 1, 0.014',file=f)
        print('../data/H2OabDefaults_SEAwater.txt',file=f) #NB - need to put a double backslash where actually want a backslash (escaping characters)
        print('astarDummy.txt',file=f)
        print('0, -999, -999, -999, -999, -999',file=f)
        print('-666, -999, -999, -999, -999, -999',file=f)
        print('bstarDummy.txt',file=f)
        print('dummybstar.txt',file=f)
        print('0, 0, 550, 0.01, 0',file=f)
        print('-2, 0, 550, 0.01, 0',file=f)
        print('dpf_pure_H2O.txt',file=f)
        print('dpf_Petzold_avg_particle.txt',file=f)
        print('25',file=f)
        print('405, 420, 438, 448, 459, 483, 493, 526, 536, 545,',file=f) #specially selected wavelengths
        print('549, 553, 557, 570, 585, 600, 615, 630, 645, 662,',file=f)
        print('672, 681, 703, 723, 743, 753,',file=f)
        print('0,0,0,0,0',file=f)
        print('2, 3, 30, 0, 0',file=f) #middle entry is solar angle
        print('-1, 0, 0, 29.92, 1, 80, 2.5, 15, 5, 300',file=f)
        print('5, 1.34, 20, 35, 3',file=f)
        print('0, 0',file=f)
        print('0, 3, 0, 1, 2',file=f) #depths
       #print('0, 21, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,\r\n',file=f)
       #print('12, 13, 14, 15, 16, 17, 18, 19, 20,\r\n',file=f)
        print('../data/H2OabDefaults_SEAwater.txt',file=f)
        print('1',file=f)
        print(acpcpath+'ac'+str(run)+'.txt',file=f) #directory to where the ac files are saved
        print('dummyFilteredAc9.txt',file=f)
        print(bbpcpath+'bb'+str(run)+'.txt',file=f) #directory to where the bb files are saved
        print('dummyCHLdata.txt',file=f)
        print('dummyCDOMdata.txt',file=f)
        print('dummyR.bot',file=f)
        print('dummydata.txt',file=f)
        print('dummyComp.txt',file=f)
        print('DummyIrrad.txt',file=f)
        print('..\\data\\MyBiolumData.txt',file=f)
        print('DummyRad.txt',file=f)

## Making the run list

In [77]:
with open(ifpath+'runlist.txt','w') as f:
    for run in range(0,nchl*nmss*ncdom,1):
        print('I_'+str(run)+'.txt',file=f)

# Running HydroLight

First, we need to copy the input files and runlist to the relevant place on our computer:

- the HE input files needs to be copied to `C:\HE60\run\batch`
- the runlist file needs to be copied to `C:\HE60\run\`

And then to run HydroLight, double click on the `run_el` application in `C:\HE60\frontend`. This will create output files in `C:\HE60\output\Ecolight\printout\`